In [ ]:
import pickle
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical

# Function to load the .pickle file
def load_pickle(file_name):
    with open(file_name, 'rb') as file:
        return pickle.load(file)

# Function to inspect the structure of the pickle file
def inspect_pickle_structure(model_data):
    print(f"Keys in model_data: {model_data.keys()}")
    if 'parameters' in model_data:
        print(f"Keys in 'parameters': {model_data['parameters'].keys()}")
    if 'initial_weights' in model_data:
        print(f"Length of 'initial_weights': {len(model_data['initial_weights'])}")
    if 'initial_biases' in model_data:
        print(f"Length of 'initial_biases': {len(model_data['initial_biases'])}")

# Function to create a model from the pickle file data
def create_model_from_pickle(model_data):
    # Inspect the model data structure to determine the correct keys
    if 'parameters' not in model_data:
        print("Error: 'parameters' key not found in model_data")
        return None

    # Get the layer sizes and other parameters
    try:
        layer_sizes = model_data['parameters']['layer_size']
    except KeyError:
        print("Error: 'layer_size' key not found in 'parameters'")
        return None

    weights = model_data['initial_weights']
    biases = model_data.get('initial_biases', None)

    # Initialize the model
    model = models.Sequential()

    # Add layers to the model based on the extracted sizes and weights
    for i in range(len(layer_sizes) - 1):
        # Create the layers
        if i == 0:
            model.add(layers.Dense(layer_sizes[i + 1], input_dim=layer_sizes[i], activation='relu'))
        else:
            model.add(layers.Dense(layer_sizes[i + 1], activation='relu'))

        # Check and convert the weights and biases into correct numpy arrays
        try:
            weight_array = np.array(weights[i]) if not isinstance(weights[i], np.ndarray) else weights[i]
            # Ensure weight_array is two-dimensional and has the correct shape
            if weight_array.ndim != 2 or weight_array.shape != (layer_sizes[i], layer_sizes[i + 1]):
                print(f"Warning: Weight shape mismatch at layer {i}, expected shape ({layer_sizes[i]}, {layer_sizes[i + 1]}) but got {weight_array.shape}")
                weight_array = np.zeros((layer_sizes[i], layer_sizes[i + 1]))  # Set to zeros if mismatch
        except Exception as e:
            print(f"Error while processing weights at layer {i}: {e}")
            weight_array = np.zeros((layer_sizes[i], layer_sizes[i + 1]))  # Set to zeros if error occurs

        try:
            bias_array = np.zeros(layer_sizes[i + 1]) if biases is None else np.array(biases[i])
            # Ensure bias_array is one-dimensional and has the correct shape
            if bias_array.ndim != 1 or bias_array.shape != (layer_sizes[i + 1],):
                print(f"Warning: Bias shape mismatch at layer {i}, expected shape ({layer_sizes[i + 1]}) but got {bias_array.shape}")
                bias_array = np.zeros(layer_sizes[i + 1])  # Set to zeros if mismatch
        except Exception as e:
            print(f"Error while processing biases at layer {i}: {e}")
            bias_array = np.zeros(layer_sizes[i + 1])  # Set to zeros if error occurs

        # Debug: Print the shapes of weights and biases
        print(f"Layer {i} - weights shape: {weight_array.shape}, biases shape: {bias_array.shape}")

        # Set the weights and biases to the layer
        model.layers[i].set_weights([weight_array, bias_array])

    # Add output layer
    model.add(layers.Dense(2, activation='softmax'))  # Example output layer for binary classification

    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Example loading models from pickle files
model_files = ['trained_model.pkl', 'trained_model1.pkl', 'trained_model2.pkl', 'trained_model3.pkl']
models_data = [load_pickle(file) for file in model_files]

# Inspect the structure of the first model
inspect_pickle_structure(models_data[0])

# Create the models using the loaded data
models = [create_model_from_pickle(model_data) for model_data in models_data]

# Compile and evaluate the models
for idx, model in enumerate(models):
    if model:
        print(f"Evaluating Model {idx + 1}...")
        # Dummy dataset for example purposes
        X, y = np.random.rand(1000, models_data[0]['parameters']['layer_size'][0]), np.random.randint(0, 2, 1000)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # One-hot encode the labels
        y_train = to_categorical(y_train, num_classes=2)
        y_test = to_categorical(y_test, num_classes=2)

        model.fit(X_train, y_train, epochs=10, batch_size=64)
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        print(f"Model {idx + 1} Classification Report:\n{classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))}")
    else:
        print(f"Model {idx + 1} could not be created due to errors.")
